In [1]:
from paddleocr import PaddleOCR
from glob import glob
from tqdm import tqdm
import pandas as pd
import json
import re
import sys

In [2]:
# import os
# from minio import Minio
# from dotenv import load_dotenv

# load_dotenv()

# client = Minio(
#     os.environ.get('S3_ENDPOINTS'),
#     access_key=os.environ.get('S3_ACCESS_KEY'),
#     secret_key=os.environ.get('S3_SECRET_KEY'),
#     secure=False
# )

In [3]:
ocr = PaddleOCR(lang="korean", show_log=False)

In [4]:
seongsu_list = glob("./data/semi-structured/*성수*.json")
seongsu_list

['./data/semi-structured/성수 데이트 맛집.json',
 './data/semi-structured/성수 회식 맛집.json']

In [5]:
apgujeong_list = glob("./data/semi-structured/*압구정*.json")
apgujeong_list

['./data/semi-structured/압구정 회식 맛집.json',
 './data/semi-structured/압구정 데이트 맛집.json']

In [6]:
def image_text(url):
    #url 크롤링 시점과 이미지 크롤링 시점이 달라서 그 사이에 원문의 이미지를 지워버린 경우 http 404에러 발생
    try :
        ocr_text = ocr.ocr(url, cls=False)
        tmp = ' '.join(t[1][0] for t in ocr_text[0])
        #특수기호 제거
        pattern = r'[^a-zA-Z가-힣]'
        tmp = re.sub(pattern=pattern, repl=' ', string=tmp)
        result = tmp
    except :
        result = ""

    return result

In [7]:
def image_data(row: dict, search_word: str) -> dict:
    template = {
        'id': '.'.join(row['url'].split('/')[-2:]),
        'search_word': search_word,
        'title': row['title'],
        'url': row['url'],
        'blog_name': row['blog_name'],
        'date': row['date']
    }
    template['content'] = '\n\n'.join([c['content'] for c in row['contents'] if c['info'] == 'text'])
    template['content_len'] = len(template['content'])

    #이미지도메인 중 제외될 도메인(기본이미지, 에디터이미지, 썸네일, 지도 등)
    exceptlist = ['https://simg', 'https://edit','https://ssl.','https://map.','https://dthu']
    #이미지 개수
    template['image_cnt'] = len([c for c in row['contents'] if c['info'] == 'img' and c['content'][:12] not in exceptlist])
    #이미지 OCR텍스트 추출
    template['image_ocr_text_paddle'] = '@@'.join([image_text(c['content']) for c in row['contents'] if c['info'] == 'img' and c['content'][:12] not in exceptlist])

#     print(template['id'], template['search_word'], template['url'], template['image_cnt'], template['image_ocr_text_paddle'])
    return template

In [ ]:
#결과 저장
#결과 저장
df = []
for path in seongsu_list:
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    fn = path.split('/')[-1].split('.')[0]
    
    res = []
    for i, d in enumerate(data):
        print(f'{i}/{len(data)}')
        res.append(image_data(d, fn))
        
    df.append(res)

    df = sum(df, [])
    df = pd.DataFrame(df)
    df = df[df.content_len > 0].drop_duplicates('id').reset_index(drop=True)
    df.to_csv('./data/ocr_data/'+fn+'_paddleocr.csv', encoding='utf-8')

0/985


100%|██████████| 288k/288k [00:00<00:00, 25.2MiB/s]
100%|██████████| 115k/115k [00:00<00:00, 16.2MiB/s]
100%|██████████| 165k/165k [00:00<00:00, 14.4MiB/s]
100%|██████████| 215k/215k [00:00<00:00, 17.3MiB/s]
100%|██████████| 179k/179k [00:00<00:00, 14.2MiB/s]
100%|██████████| 217k/217k [00:00<00:00, 12.2MiB/s]
100%|██████████| 224k/224k [00:00<00:00, 11.1MiB/s]
100%|██████████| 238k/238k [00:00<00:00, 7.33MiB/s]
100%|██████████| 159k/159k [00:00<00:00, 11.0MiB/s]
100%|██████████| 212k/212k [00:00<00:00, 12.8MiB/s]
100%|██████████| 36.1k/36.1k [00:00<00:00, 16.8MiB/s]
100%|██████████| 31.8k/31.8k [00:00<00:00, 14.5MiB/s]
100%|██████████| 33.1k/33.1k [00:00<00:00, 22.8MiB/s]
100%|██████████| 37.0k/37.0k [00:00<00:00, 20.2MiB/s]
100%|██████████| 225k/225k [00:00<00:00, 12.1MiB/s]
100%|██████████| 262k/262k [00:00<00:00, 10.0MiB/s]
100%|██████████| 251k/251k [00:00<00:00, 15.1MiB/s]
100%|██████████| 450k/450k [00:00<00:00, 19.3MiB/s]
100%|██████████| 451k/451k [00:00<00:00, 24.8MiB/s]
100%

1/985


100%|██████████| 195k/195k [00:00<00:00, 17.3MiB/s]
100%|██████████| 195k/195k [00:00<00:00, 11.4MiB/s]
100%|██████████| 175k/175k [00:00<00:00, 8.51MiB/s]
100%|██████████| 144k/144k [00:00<00:00, 14.2MiB/s]
100%|██████████| 148k/148k [00:00<00:00, 16.5MiB/s]
100%|██████████| 49.7k/49.7k [00:00<00:00, 13.0MiB/s]
100%|██████████| 41.7k/41.7k [00:00<00:00, 14.0MiB/s]
100%|██████████| 189k/189k [00:00<00:00, 14.1MiB/s]
100%|██████████| 117k/117k [00:00<00:00, 11.9MiB/s]
100%|██████████| 209k/209k [00:00<00:00, 14.6MiB/s]
100%|██████████| 47.7k/47.7k [00:00<00:00, 15.4MiB/s]
100%|██████████| 48.4k/48.4k [00:00<00:00, 9.64MiB/s]
100%|██████████| 158k/158k [00:00<00:00, 14.2MiB/s]
100%|██████████| 176k/176k [00:00<00:00, 13.6MiB/s]
100%|██████████| 109k/109k [00:00<00:00, 15.3MiB/s]
100%|██████████| 139k/139k [00:00<00:00, 13.8MiB/s]
100%|██████████| 121k/121k [00:00<00:00, 14.2MiB/s]
100%|██████████| 32.9k/32.9k [00:00<00:00, 14.1MiB/s]
100%|██████████| 30.5k/30.5k [00:00<00:00, 13.9MiB/s]


2/985


...

100%|██████████| 81.0k/81.0k [00:00<00:00, 12.6MiB/s]
100%|██████████| 94.5k/94.5k [00:00<00:00, 16.8MiB/s]
100%|██████████| 90.9k/90.9k [00:00<00:00, 20.5MiB/s]
100%|██████████| 86.5k/86.5k [00:00<00:00, 11.2MiB/s]
100%|██████████| 87.6k/87.6k [00:00<00:00, 12.8MiB/s]
100%|██████████| 92.3k/92.3k [00:00<00:00, 16.4MiB/s]
100%|██████████| 85.0k/85.0k [00:00<00:00, 15.0MiB/s]
100%|██████████| 97.8k/97.8k [00:00<00:00, 13.0MiB/s]
100%|██████████| 97.9k/97.9k [00:00<00:00, 12.5MiB/s]
100%|██████████| 97.8k/97.8k [00:00<00:00, 12.6MiB/s]
100%|██████████| 107k/107k [00:00<00:00, 12.0MiB/s]
100%|██████████| 93.4k/93.4k [00:00<00:00, 12.7MiB/s]
100%|██████████| 84.7k/84.7k [00:00<00:00, 19.7MiB/s]
100%|██████████| 77.3k/77.3k [00:00<00:00, 9.77MiB/s]
100%|██████████| 103k/103k [00:00<00:00, 13.6MiB/s]
100%|██████████| 92.6k/92.6k [00:00<00:00, 19.9MiB/s]
100%|██████████| 106k/106k [00:00<00:00, 13.8MiB/s]
100%|██████████| 8.01k/8.01k [00:00<00:00, 28.9MiB/s]
100%|██████████| 120k/120k [00:00<

674/985


100%|██████████| 126k/126k [00:00<00:00, 14.4MiB/s]
100%|██████████| 121k/121k [00:00<00:00, 21.3MiB/s]
100%|██████████| 103k/103k [00:00<00:00, 10.1MiB/s]
100%|██████████| 147k/147k [00:00<00:00, 11.3MiB/s]
100%|██████████| 185k/185k [00:00<00:00, 14.9MiB/s]
100%|██████████| 155k/155k [00:00<00:00, 11.5MiB/s]
100%|██████████| 132k/132k [00:00<00:00, 9.11MiB/s]
100%|██████████| 215k/215k [00:00<00:00, 15.0MiB/s]
100%|██████████| 108k/108k [00:00<00:00, 10.2MiB/s]
100%|██████████| 216k/216k [00:00<00:00, 11.5MiB/s]
100%|██████████| 163k/163k [00:00<00:00, 14.5MiB/s]
100%|██████████| 221k/221k [00:00<00:00, 17.0MiB/s]
100%|██████████| 139k/139k [00:00<00:00, 19.0MiB/s]
100%|██████████| 92.1k/92.1k [00:00<00:00, 13.1MiB/s]
100%|██████████| 98.9k/98.9k [00:00<00:00, 16.1MiB/s]
100%|██████████| 190k/190k [00:00<00:00, 18.4MiB/s]


675/985


100%|██████████| 21.9k/21.9k [00:00<00:00, 18.9MiB/s]
100%|██████████| 57.0k/57.0k [00:00<00:00, 8.50MiB/s]
100%|██████████| 51.7k/51.7k [00:00<00:00, 13.9MiB/s]
100%|██████████| 270k/270k [00:00<00:00, 12.8MiB/s]
100%|██████████| 317k/317k [00:00<00:00, 14.3MiB/s]
100%|██████████| 185k/185k [00:00<00:00, 19.2MiB/s]
100%|██████████| 47.0k/47.0k [00:00<00:00, 3.64MiB/s]
100%|██████████| 24.4k/24.4k [00:00<00:00, 11.0MiB/s]
100%|██████████| 212k/212k [00:00<00:00, 16.2MiB/s]
100%|██████████| 157k/157k [00:00<00:00, 19.1MiB/s]
100%|██████████| 139k/139k [00:00<00:00, 6.78MiB/s]
100%|██████████| 232k/232k [00:00<00:00, 19.9MiB/s]
100%|██████████| 66.5k/66.5k [00:00<00:00, 16.3MiB/s]
100%|██████████| 66.0k/66.0k [00:00<00:00, 14.4MiB/s]
100%|██████████| 151k/151k [00:00<00:00, 11.9MiB/s]
100%|██████████| 142k/142k [00:00<00:00, 15.1MiB/s]
100%|██████████| 259k/259k [00:00<00:00, 9.31MiB/s]
100%|██████████| 210k/210k [00:00<00:00, 13.0MiB/s]
  0%|          | 0.00/45.3k [00:00<?, ?iB/s]